<a href="https://colab.research.google.com/github/yecatstevir/teambrainiac/blob/main/source/AccuracyMeasures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')#, force_remount = True)

Mounted at /content/gdrive


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
# Clone the entire repo.
!git clone -l -s https://github.com/yecatstevir/teambrainiac.git
# Change directory into cloned repo
%cd teambrainiac/source
!ls

Cloning into 'teambrainiac'...
remote: Enumerating objects: 635, done.
remote: Counting objects: 100% (635/635), done.
remote: Compressing objects: 100% (451/451), done.
remote: Total 635 (delta 386), reused 343 (delta 168), pack-reused 0
Receiving objects: 100% (635/635), 68.38 MiB | 11.01 MiB/s, done.
Resolving deltas: 100% (386/386), done.
/content/teambrainiac/source
AccuracyMeasures.ipynb	  models
analysis.py		  process.py
cross_validation.py	  SingleSubjectSVM.ipynb
data			  SubjectVisualization_Models_ZNORM.ipynb
DL			  SVM_Group_Adolescent_Whole_brain.ipynb
Explore_data.ipynb	  SVM_Group_YA_Whole_brain.ipynb
explore.py		  train.py
Group_All_MASK_SVM.ipynb  utils.py
helper			  VisualizationPlayground.ipynb
Images			  Visualize_Data.ipynb
__init__.py


In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving path_config.py to path_config.py
User uploaded file "path_config.py" with length 228 bytes


In [11]:
# Import libraries
!pip install boto3 nilearn nibabel
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, auc, recall_score, precision_score,roc_curve,f1_score
import pickle
from utils import *
from sklearn.svm import SVC
import numpy as np
import pandas as pd

In [6]:
path = "data/data_path_dictionary.pkl"
data_path_dict = open_pickle(path)
subject_ids = data_path_dict['subject_ID']

In [14]:
def get_accuracy_scores(path,get_ind=True,model_dict={}):
  subj_acc = []
  for subj in subject_ids:
    #subj_data = model_dict[subj]
    if get_ind:
      destination_path = f"{path}{subj}.pkl"
      print(destination_path)
      model_dict = access_load_data(destination_path,False)
    clf = model_dict[subj]['model']
    X_train = model_dict[subj]['X_train']
    y_train = model_dict[subj]['y_train']
    X_test = model_dict[subj]['X_test'] 
    y_test = model_dict[subj]['y_test']
    if 'X_val' in model_dict[subj].keys():
      X_val = model_dict[subj]['X_val']
      y_val = model_dict[subj]['y_val']
      y_val_predicts = clf.predict(X_val)
      val_acc = accuracy_score(y_val,y_val_predicts)
      fpr,tpr,thresholds = roc_curve(y_val,y_val_predicts)
      val_auc = auc(fpr,tpr)
      val_precision = precision_score(y_val,y_val_predicts)
      val_recall = recall_score(y_val,y_val_predicts)
      val_f1 = f1_score(y_val,y_val_predicts)
    else:
      y_val_predicts = 0
      val_acc = 0
    train_predicts = clf.predict(X_train) 
    y_test_predicts = clf.predict(X_test)
    tr_acc = accuracy_score(y_train,train_predicts)   
    test_acc = accuracy_score(y_test,y_test_predicts)
    fpr,tpr,thresholds = roc_curve(y_test,y_test_predicts)
    test_auc = auc(fpr,tpr)
    test_precision = precision_score(y_test,y_test_predicts)
    test_recall = recall_score(y_test,y_test_predicts)
    test_f1 = f1_score(y_test,y_test_predicts)
    subj_list = [subj,tr_acc,val_acc,val_auc,val_precision,val_recall,val_f1, test_acc,test_auc,test_precision,test_recall,test_f1,'RUNS','PSCNORM']
    subj_acc.append(subj_list)
  return subj_acc

In [18]:
sub_acc_pd = get_accuracy_scores("models/single/one_run_model_znorm_",get_ind=True)


models/single/one_run_model_znorm_10004_08693.pkl


EOFError: ignored

In [ ]:
sub_acc_df = pd.DataFrame(sub_acc_pd, columns = ["Subject","TrainAcc","ValAcc","ValAUC","ValPrecision","ValRecall","ValF1",
                                                 "TestAcc","TestAUC","TestPrecision","TestRecall","TestF1","NormBy","DataType"])
path = "/content/drive/My Drive/data/one_run_nonorm_metrics_all.csv"
sub_acc_df.to_csv(path)

In [ ]:
acc_df_psconly = pd.read_csv("/content/drive/My Drive/data/one_run_pcs_metrics_all.csv")
acc_df_znorm_only = pd.read_csv("/content/drive/My Drive/data/one_run_znorm_metrics_all.csv")
acc_df_znorm_psc = pd.read_csv("/content/drive/My Drive/data/one_run_psc_znorm_metrics_all.csv")
acc_df_nonorm = sub_acc_df
acc_df_dict = {}
acc_df_dict['acc_df_psconly'] = acc_df_psconly
acc_df_dict['acc_df_znorm_only'] = acc_df_znorm_only
acc_df_dict['acc_df_znorm_psc'] = acc_df_znorm_psc
acc_df_dict['acc_df_nonorm'] = acc_df_nonorm


In [ ]:
object_name = 'single_subj_acc_measures.pkl'
upload = s3_upload(acc_df_dict,object_name,"pickle")

upload complete for single_subj_acc_measures.pkl
